# Defacing pre-registration - Statistical analysis in R

## Load simulated data

In [2]:
df <- read.table(file = 'SimulatedDefacedRatings.csv', sep = ",", header = TRUE, stringsAsFactors = TRUE)

The simulated data were generated by running the `SimulateDefacedRatings.ipnyb` notebook.

## Continuation ratio mixed effects regression

The ratings are ordinal, i.e. a categorical variable with a natural ordering of its levels. In order to model raters’ variabilities, we will use mixed effects regression from the GLMMadaptive package in R . This analysis is inspired from https://drizopoulos.github.io/GLMMadaptive/articles/Ordinal_Mixed_Models.html.

The backward formulation is commonly used when progression through ratings from excluded, poor, good, excellent, is represented by increasing integer values, and interest lies in estimating the odds of better quality compared to worst quality. The forward formulation specifies that subjects have to ‘pass through’ one category to get to the next one. The backward formulation is thus suitable for our analysis.

An advantage of the continuation ratio model is that its likelihood can be easily re-expressed such that it can be fitted with software the fits (mixed effects) logistic regression. This formulation requires a couple of data management steps creating separate records for each measurement, and suitably replicating the corresponding rows of the design matrices Xi and Zi. In addition, a new ‘cohort’ variable is constructed denoting at which category the specific measurement of i-th subject belongs. An extra advantage of this formulation is that we can easily evaluate if specific covariates satisfy the ordinality assumption (i.e., that their coefficients are independent of the category k) by including into the model their interaction with the ‘cohort’ variable and testing its significance.


In [4]:
library(GLMMadaptive)

## Data set-up and calculation of marginal probabilites from a continuation ratio model
cr_vals <- cr_setup(df$ratings, direction = "backward")
cr_data <- df[cr_vals$subs, ]
cr_data$ratings_new <- cr_vals$y
cr_data$cohort <- cr_vals$cohort

## Fits generalized linear mixed effects model
fm <- mixed_model(ratings_new ~ cohort + defaced + rater, random = ~ 1 | rater, data = cr_data, family = binomial(), na.action = na.exclude)
fm


Call:
mixed_model(fixed = ratings_new ~ cohort + defaced + rater, random = ~1 | 
    rater, data = cr_data, family = binomial(), na.action = na.exclude)


Model:
 family: binomial
 link: logit 

Random effects covariance matrix:
                 StdDev
(Intercept) 0.009017118

Fixed effects:
      (Intercept)     cohorty<=good cohorty<=excluded   defacedoriginal 
    -0.9129735766      0.3668093138      0.6392333953     -0.0006134864 
           rater2            rater3            rater4            rater5 
    -0.1263234751     -0.0688977084     -0.1184201225      0.0001946318 
           rater6            rater7            rater8            rater9 
    -0.0242123897     -0.2368901397     -0.3441942007     -0.3221685507 
          rater10           rater11           rater12 
    -0.3926695910     -0.1975251922     -0.2950198344 

log-Lik: -19062.91


`random = ~ 1 | rater` enables to account for between-raters variability. This formula lets the intercept take a different value for each rater.

### Relaxing CR assumption

We can relax the ordinality assumption for the defaced variable, namely, allowing that the effect of defaced is different for each of the response categories of our ordinal outcome y. This can be achieved by simply including the interaction term between the defaced and cohort variables and/or the interaction term between the rater and the cohort variables.

In [8]:
gm <- mixed_model(ratings_new ~ cohort * defaced + rater, random = ~ 1 | rater, data = cr_data, family = binomial())
gm


Call:
mixed_model(fixed = ratings_new ~ cohort * defaced + rater, random = ~1 | 
    rater, data = cr_data, family = binomial())


Model:
 family: binomial
 link: logit 

Random effects covariance matrix:
                StdDev
(Intercept) 0.00835499

Fixed effects:
                 (Intercept)                cohorty<=good 
                 -1.27016209                   0.37607078 
               cohorty<=poor               defaceddefaced 
                  1.10847654                   0.54198717 
                      rater2                       rater3 
                 -0.06513242                  -0.05340367 
                      rater4                       rater5 
                  0.02817786                   0.17916487 
                      rater6                       rater7 
                  0.11767470                   0.14844202 
                      rater8                       rater9 
                  0.25101273                   0.19715950 
                     rat

In [35]:
#gm2 <- mixed_model(ratings_new ~ cohort * defaced + cohort * rater, random = ~ 1 | rater, data = cr_data, family = binomial())
#gm2


Call:
mixed_model(fixed = ratings_new ~ cohort * defaced + rater, random = ~1 | 
    rater, data = cr_data, family = binomial())


Model:
 family: binomial
 link: logit 

Random effects covariance matrix:
                 StdDev
(Intercept) 0.008354982

Fixed effects:
                  (Intercept)                 cohorty<=good 
                  -0.72818126                    0.10594841 
                cohorty<=poor               defacedoriginal 
                   1.19284550                   -0.54196754 
                       rater2                        rater3 
                  -0.06513242                   -0.05340367 
                       rater4                        rater5 
                   0.02817786                    0.17916487 
                       rater6                        rater7 
                   0.11767470                    0.14844202 
                       rater8                        rater9 
                   0.25101273                    0.19715950

To test whether these extensions are required we can perform a likelihood ratio test using the anova() method:

In [37]:
anova(fm,gm)


        AIC      BIC   log.Lik   LRT df p.value
fm 37740.76 37748.52 -18854.38                 
gm 37710.82 37719.54 -18837.41 33.94  2 <0.0001


In [36]:
#anova(gm, gm2)


         AIC      BIC   log.Lik   LRT df p.value
gm  37710.82 37719.54 -18837.41                 
gm2 37730.15 37749.54 -18825.07 24.67 22  0.3132


**Discussion** The first test rejects the null hypothesis meaning that defaced doesn't satisfy the continuous ratio assumption => the addition of the interaction term cohort x defaced is thus necessary. However, the second test doesn't reject the null hypothesis, meaning that rater satisfied the continuation ratio assumption, thus the interaction term cohort x rater is not necessary.

gm is our final model.

### Effect plot of conditional probabilities

In [55]:
library(lattice)

#Extract data necessary to plot
nDF <- with(cr_data, expand.grid(cohort = levels(cohort), defaced = levels(defaced), 
                                 rater = rater))

plot_data <- effectPlotData(gm, nDF, direction="backward")

#Plot
expit <- function (x) exp(x) / (1 + exp(x))
my_panel_bands <- function(x, y, upper, lower, fill, col, subscripts, ..., font, 
                           fontface) {
    upper <- upper[subscripts]
    lower <- lower[subscripts]
    panel.polygon(c(x, rev(x)), c(upper, rev(lower)), col = fill, border = FALSE, ...)
}


xyplot(expit(pred) ~ rater | defaced, group = cohort, data = plot_data, 
       upper = expit(plot_data$upp), low = expit(plot_data$low), type = "l",
       panel = function (x, y, ...) {
           panel.superpose(x, y, panel.groups = my_panel_bands, ...)
           panel.xyplot(x, y, lwd = 2,  ...)
       }, xlab = "Rater", ylab = "Continuation Ratio Probabilities")

Warning message in model.frame.default(termsX, newdata, xlev = .getXlevels(termsX, :
“variable 'rater' is not a factor”


ERROR: Error in X %*% betas: non-conformable arguments


### Effect plot of marginal probabilities

The effect plot of the previous section depicts the conditional probabilities according to the backward formulation of the continuation ratio model. However, it is easier to understand the marginal probabilities of each category.

In [53]:
#Extract data for plot
plot_data_m <- effectPlotData(fm, nDF, CR_cohort_varname = "cohort", 
                              direction = "backward")
#Plot
key <- list(space = "top", rep = FALSE,
            text = list(levels(DF$y)[1:2]),
            lines = list(lty = c(1, 1), lwd = c(2, 2), col = c("#0080ff", "#ff00ff")),
            text = list(levels(DF$y)[3:4]),
            lines = list(lty = c(1, 1), lwd = c(2, 2), col = c("darkgreen", "#ff0000")))

xyplot(expit(pred) ~ time | sex, group = ordinal_response, data = plot_data_m, 
       upper = expit(plot_data_m$upp), low = expit(plot_data_m$low), type = "l",
       panel = function (x, y, ...) {
           panel.superpose(x, y, panel.groups = my_panel_bands, ...)
           panel.xyplot(x, y, lwd = 2, ...)
       }, xlab = "Follow-up time", ylab = "Marginal Probabilities", key = key)

Warning message in model.frame.default(termsX, newdata, xlev = .getXlevels(termsX, :
“variable 'rater' is not a factor”


ERROR: Error in X %*% betas: non-conformable arguments


### Ordered logistic regression

Test just ordered logistic regression before increasing the complexity using mixed_model. Inspired from https://stats.oarc.ucla.edu/r/dae/ordinal-logistic-regression/

In [4]:
library(MASS)
m <- polr(ratings ~ defaced + rater, data = df, Hess=TRUE, method = "logistic")
m


Attaching package: ‘MASS’


The following object is masked from ‘package:GLMMadaptive’:

    negative.binomial




Call:
polr(formula = ratings ~ defaced + rater, data = df, Hess = TRUE, 
    method = "logistic")

Coefficients:
defacedoriginal           rater 
    -0.54861742      0.05227948 

Intercepts:
 excluded|poor      poor|good good|excellent 
    -1.3730969     -0.1424015      0.9006451 

Residual Deviance: 37781.85 
AIC: 37791.85 

One way to calculate a p-value in this case is by comparing the t-value against the standard normal distribution, like a z test. Of course this is only true with infinite degrees of freedom, but is reasonably approximated by large samples, becoming increasingly biased as sample size decreases

In [5]:
## Store table
ctable <- coef(summary(m))

## calculate and store p values
p <- pnorm(abs(ctable[, "t value"]), lower.tail = FALSE) * 2

## combined table
(ctable <- cbind(ctable, "p value" = p))

,Value,Std. Error,t value,p value
defacedoriginal,-0.54861742,0.03072127,-17.857904,2.508829e-71
rater,0.05227948,0.00443805,11.779831,4.959221e-32
excluded|poor,-1.37309692,0.03900182,-35.205974,1.620050e-271
poor|good,-0.14240145,0.03691166,-3.857899,1.143660e-04
good|excellent,0.90064511,0.03770962,23.883699,4.524274e-126


We can also get confidence intervals for the parameter estimates. These can be obtained either by profiling the likelihood function or by using the standard errors and assuming a normal distribution. Note that profiled CIs are not symmetric (although they are usually close to symmetric). If the 95% CI does not cross 0, the parameter estimate is statistically significant.

In [37]:
(ci <- confint(m)) # default method gives profiled CIs
confint.default(m) # CIs assuming normality

Waiting for profiling to be done...



,2.5 %,97.5 %
defacedoriginal,-0.60886781,-0.48844059
rater,0.04358562,0.06098284


,2.5 %,97.5 %
defacedoriginal,-0.60882999,-0.4884048
rater,0.04358106,0.0609779


Both the p-values and the confidence interval indicate that both the defacing and rater predictors improve the fit of the model.

#### Verifying proportional odds assumption

One of the assumptions underlying ordinal logistic (and ordinal probit) regression is that the relationship between each pair of outcome groups is the same. In other words, ordinal logistic regression assumes that the coefficients that describe the relationship between, say, the lowest versus all higher categories of the response variable are the same as those that describe the relationship between the next lowest category and all higher categories, etc. This is called the proportional odds assumption or the parallel regression assumption. Because the relationship between all pairs of groups is the same, there is only one set of coefficients. If this was not the case, we would need different sets of coefficients in the model to describe the relationship between each pair of outcome groups.

In [24]:
#The sf function will calculate the log odds of being greater than or equal to each value of the target variable
sf <- function(y) {
  c('Y>=1' = qlogis(mean(y >= 1)),
    'Y>=2' = qlogis(mean(y >= 2)),
    'Y>=3' = qlogis(mean(y >= 3)),
    'Y>=4' = qlogis(mean(y >= 4)))
}

#calls the function sf on several subsets of the data defined by the predictors
(s <- with(df, summary(as.numeric(ratings) ~ defaced + rater, fun=sf)))

 Length   Class    Mode 
      3 formula    call 

The table above displays the (linear) predicted values we would get if we regressed our dependent variable on our predictor variables one at a time, without the parallel slopes assumption. We can evaluate the parallel slopes assumption by running a series of binary logistic regressions with varying cutpoints on the dependent variable and checking the equality of coefficients across cutpoints. We thus relax the parallel slopes assumption to checks its tenability. To accomplish this, we transform the original, ordinal, dependent variable into a new, binary, dependent variable which is =0 if the original, ordinal dependent variable (here apply) is < some value a, and 1 if the ordinal variable is >= a.

In [29]:
glm(I(as.numeric(ratings) >= 2) ~ defaced, family="binomial", data = df)


Call:  glm(formula = I(as.numeric(ratings) >= 2) ~ defaced, family = "binomial", 
    data = df)

Coefficients:
    (Intercept)  defacedoriginal  
         1.8206          -0.7326  

Degrees of Freedom: 13919 Total (i.e. Null);  13918 Residual
Null Deviance:	    13760 
Residual Deviance: 13480 	AIC: 13480

In [28]:
glm(I(as.numeric(ratings) >= 3) ~ defaced, family="binomial", data = df)


Call:  glm(formula = I(as.numeric(ratings) >= 3) ~ defaced, family = "binomial", 
    data = df)

Coefficients:
    (Intercept)  defacedoriginal  
         0.4409          -0.4593  

Degrees of Freedom: 13919 Total (i.e. Null);  13918 Residual
Null Deviance:	    19150 
Residual Deviance: 18970 	AIC: 18970

When defaced = 'defaced', the difference between the predicted value for apply >=2 and apply >=3 is roughly 2 (1.82 + 0.44 = 2.26). For defaced = 'original', the difference in predicted values for apply >= 2 and apply >=3 is roughly 1 ((1.82-0.73)+(0.44-0.46) = 1.07). This suggests that the parallel slopes assumption is not reasonable.

### Test significance of predictors

We test significance of the predictors using Wald test. Specifically, we want to test whether the coefficient associated to the fixed effect defacing is significantly non-zero. Note that the use of this test is possible because we have a balanced, nested GLMM. Inspired from https://www.statology.org/wald-test-in-r/.


In [ ]:
library(aod)

#perform Wald Test to determine if defacing variables is zero
#This is copied from an example using lm but doesn't work for mixed_model
wald.test(Sigma = vcov(fm), b = coef(fm), Terms = 1)

If p-value > 0.05, it means we fail to reject the null hypothesis, meaning defacing coefficient is approximately zero. Thus you can drop that predictor from the model, because it doesn't significantly improve the fit of the model.